In [31]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')


In [32]:
#BASES
colombia = pd.read_excel('Pico_marcacion_Colombia.xlsx') 
colombia.drop(columns='Country',inplace=True)
colombia['Pais'] = 'Colombia'

peru     = pd.read_excel('Pico_marcacion_Peru.xlsx')
peru.drop(columns='Country',inplace=True)
peru['Pais'] = 'Peru'

ecuador  = pd.read_excel('Pico_marcacion_Ecuador.xlsx')
ecuador.drop(columns='Country',inplace=True)
ecuador['Pais'] = 'Ecuador'

In [33]:
ecuador['Date'].value_counts()

2024-01-10 02:27:54.0    3
2024-01-10 02:58:55.0    2
2024-01-10 02:42:49.0    2
2024-01-10 02:47:28.0    2
2024-01-10 03:13:51.0    2
                        ..
2024-01-10 21:02:12.0    1
2024-01-10 21:03:12.0    1
2024-01-10 21:11:06.0    1
2024-01-10 21:11:40.0    1
2024-01-08 05:00:00.0    1
Name: Date, Length: 1455, dtype: int64

In [34]:
def resultados(df):

   df = df[df[df.columns[6]] == 0]
   df[df.columns[6]] = np.where(df[df.columns[6]]==0,1,0)
   df.columns = df.columns.str.strip()


   #Fecha Pico
   df['Fecha pico'] = pd.to_datetime(df['Date'])

   df['Dia'] = df['Fecha pico'].dt.day
   df['Mes'] = df['Fecha pico'].dt.month
   df['Ano'] = df['Fecha pico'].dt.year

   meses_ingles_a_espanol = {
   1: 'Ene', 2: 'Feb', 3: 'Mar', 4: 'Abr',
   5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Ago',
   9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dic'}

   df['Mes'] = df['Mes'].map(meses_ingles_a_espanol)
   df['Dia'] = [str(x) for x in df['Dia']]
   df['Ano'] = [str(x) for x in df['Ano']]
   
   df['Fecha pico'] = df['Dia'] + '-' + df['Mes'] + '-' + df['Ano']

   #Agrupamiento
   df = df.groupby(by=['Pais','Fecha pico'])[['Llamados a la acción', 'Sanciones Judiciales',
      'Insultos', 'No es Xenofobia', 'Impact', 'Impressions']].sum()
   

   df.rename(columns = {'No es Xenofobia':'Xenofobia',
                        'Impact':'Total impacto de publicaciones',
                        'Impressions':'Total Impresiones (veces que se vieron estos mensajes)',
                        'Sanciones Judiciales':'Sanciones'},inplace=True)
   
   df[['% Llamados a la acción','% Sanciones Judiciales','% Insultos']] = 0     

   df.reset_index(inplace=True)
   
   #Calculo de estadisticos

   df['% Llamados a la acción'] = [(0 if a == 0 else a / b * 100) for a, b in zip(df['Llamados a la acción'], df['Xenofobia'])]
   df['% Sanciones Judiciales'] = [(0 if a == 0 else a / b * 100) for a, b in zip(df['Sanciones'], df['Xenofobia'])]
   df['% Insultos'] = [(0 if a == 0 else a / b * 100) for a, b in zip(df['Insultos'], df['Xenofobia'])]


   #Organizado

   df = df[['Pais','Llamados a la acción', 'Sanciones', 'Insultos','Fecha pico',
            'Total impacto de publicaciones','Total Impresiones (veces que se vieron estos mensajes)','Xenofobia', 
            '% Llamados a la acción', '% Sanciones Judiciales', '% Insultos']]

   df[['% Llamados a la acción', '% Sanciones Judiciales', '% Insultos']] = (df[['% Llamados a la acción', '% Sanciones Judiciales', '% Insultos']]).applymap(lambda x: f'{x:.0f}%')

   # df['Total impacto de publicaciones']  = df['Total impacto de publicaciones'].astype(str)
   df['Total impacto de publicaciones']  = df['Total impacto de publicaciones'].round(2)

   return df

In [35]:
# Resultados

colombia,peru,ecuador = resultados(colombia),resultados(peru),resultados(ecuador)

picos = pd.concat([colombia,peru,ecuador])

picos.to_csv('pico_xenofobia_limpio.csv',index=False)